# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
sys.path.append("../../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api
from Clust.clust.ML.tool import meta

db_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


# 1. set param from Front End

In [2]:
# feature list와 data_y_flag는 model_info에서 받아와야 합니다.
# regression
param1 = {
    'model_name':'regression_energy_cleanLevel4_trainX_regression_GRU_rg_',
    'ingestion_param_y':{
        "feature_list": "None"
    },
    'dataX_src': 'local'
}

# forecasting
param2 = {
    'model_name':'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_GRU_rg_',
    'ingestion_param_y':{
        "feature_list": "None"
    },
    'dataX_src': 'local'
}

# classification
param3 = {
    'model_name':'classification_actionPattern_cleanLevel0_trainX_classification_GRU_cf_',
    'ingestion_param_y':{
        "feature_list": "None"
    },
    'dataX_src': 'local'
}

### (Optional) Get np.array data for inference

In [3]:
import numpy as np
np_data = np.load('infer_classification_array.npy') # infer_regression_array , infer_forecasting_array , infer_classification_array
infer_np = np_data[8]
infer_np.shape

(128, 9)

# 2. Backend

In [5]:
# 1. Backend Parameter Setting
params = param3

# chage tpye string to bool -> ex) 'true' -> True
params = ML_api.chagne_type_str_to_bool(params)

In [6]:
# 2. get model meta
model_name = params['model_name']
meta_file_name = "./meta.json"
#model_meta_db_style ='document'

# for testing mongodb model
#model_meta = meta.read_model_meta_from_mongodb(mongodb_client, 'model', 'meta', model_name)
# for testing local model
model_meta = meta.read_model_meta_from_local(meta_file_name, model_name)
# model_meta = meta.read_model_meta_by_db_style(model_meta_db_style, meta_file_name, model_name)

# scaler info for inference data preparation
params['scaler_param'] = model_meta['scaler_param']

# model & target data info for ML inference
params['model_info'] = model_meta['model_info']
params['ingestion_param_y']['feature_list'] = model_meta['ingestion_param_y']['feature_list']

In [7]:
params['model_info']['train_parameter']['batch_size'] = 1

In [8]:
# 3. Data Preparation
# make inference data X, scaler y
infer_X, scaler_y = ML_api.infer_data_preparation(params, infer_np)

In [10]:
infer_X.shape

(128, 9)

In [11]:
# model inference
prediction_result = ML_api.ML_inference(params, infer_X, scaler_y)

inference data shape: torch.Size([1, 128, 9])

Start inference

tensor([[10.6109, -7.9217, -5.2556, -5.2408, -5.2841, -5.2798]])
** Dimension of result for inference dataset = (1,)


In [12]:
prediction_result

,value
0,0.0
